## spark sql

1-create df

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

df = spark.read.format("csv").option("header", "false").option("inferSchema", "true").option("sep", "\t").load("ngram.csv")
df = df.toDF('bigram','year','count', 'pages','books')

df.show(10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/11 20:49:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
+--------+----+-----+-----+-----+
only showing top 10 rows



2-tmp table

In [5]:
df.createOrReplaceTempView("ngram")

3.1-return all bigrams where the Count is greater than five.

In [ ]:
# sql
result1 = spark.sql("Select * from ngram where count > 5")
result1.show(10)

+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
+--------+----+-----+-----+-----+
only showing top 10 rows



In [12]:
# sparkSql
df.filter("count > 5").select("bigram").show(10)

+--------+
|  bigram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+
only showing top 10 rows



3.2-return the total number of bigrams for each year.

3.3-return the bigrams with the highest Count for each year.

3.4-return all bigrams that appeared in 20 different years.

3.5-return all bigrams that contain the character '!' in the first part and the character '9' in the second part (the two parts are separated by a space).

3.6-return the bigrams that appeared in all the years present in the dataset.

3.7-return the total number of pages and books in which each bigram appeared for each available year, sorted in alphabetical order.

3.8-return the total number of distinct bigrams for each year, sorted in descending order of the year.